In [45]:
%matplotlib inline

In [46]:
# Dependencies and Setup
import gmaps
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
import time
import api_keys
from ipywidgets.embed import embed_minimal_html

# Affordable Housing Inventory (AHI)

In [47]:
# File to Load
AHI_to_load = "Resources/AHI_Affordable_Housing_Inventory.csv"
# Read the Affordable Housing Inventory Data
AHI_to_load = pd.read_csv("Resources/AHI_Affordable_Housing_Inventory.csv")
AHI_to_load.head()

,AHI #,Project ID,Project Name,Owner,Developer,Address,Zip Code,Council District,Kirwan Opportunity Index,Total Units,...,CIP,UNO TF,Total City Funding,LIHTC,Property Management Company,Phone Number,Website,Longitude,Latitude,Location
0,2,3225,M Station,"Foundation Communities, Inc.","Foundation Communities, Inc.",2906 E Martin Luther King Jr Boulevard,78722,1.0,Very High,150,...,$0.00,$0.00,"$2,000,000.00",9%,Foundation Communities,512-474-6767,http://foundcom.org/housing/our-austin-communi...,-97.708414,30.282007,"(30.2820072186666, -97.7084137218687)"
1,3,3226,Garden Terrace Phase II,Garden Terrace Housing Corporation,"Foundation Communities, Inc.",1015 W William Cannon Drive,78745,2.0,Low,15,...,$0.00,$0.00,"$500,000.00",NaN,Foundation Communities,512-416-8300,http://foundcom.org/housing/our-austin-communi...,-97.793427,30.198641,"(30.1986408937312, -97.7934265318501)"
2,4,3227,LifeWorks Transitional Living Project,Lifeworks,Lifeworks,3710 S 2nd Street,78704,3.0,Very Low,12,...,$0.00,$0.00,"$300,000.00",NaN,Lifeworks,512-735-2100,http://www.lifeworksaustin.org/housing/,-97.771020,30.229837,"(30.2298368209301, -97.7710197443868)"
3,5,3228,Cornerstone,Mary Lee Community,Mary Lee Foundation,1322 Lamar Square Drive,78704,5.0,Very Low,30,...,$0.00,$0.00,"$1,009,502.00",NaN,Mary Lee Foundation,512-443-5777,http://www.maryleefoundation.org/programs-and-...,-97.764224,30.255306,"(30.2553062066707, -97.7642238885743)"
4,6,3229,The Willows,Mary Lee Community,Mary Lee Foundation,1330 Lamar Square Drive,78704,5.0,Moderate,64,...,$0.00,$0.00,"$2,250,000.00",NaN,Mary Lee Foundation,512-443-5777,http://www.maryleefoundation.org/programs-and-...,-97.765039,30.254456,"(30.2544562030313, -97.765039314016)"


In [48]:
list(AHI_to_load.columns.values)

['AHI #',
 'Project ID',
 'Project Name',
 'Owner',
 'Developer',
 'Address',
 'Zip Code ',
 'Council District',
 'Kirwan Opportunity Index',
 'Total Units',
 'Total Affordable Units',
 'Total PSH Units ',
 '1/4 Mile from High Frequency Transit',
 '3/4 Mile from Local Transit',
 'Unit Type',
 'Tenure',
 'Status',
 'Affordability Start Year',
 'Affordability Expiration Year ',
 'Affordability Period',
 'Fee in Lieu Status',
 'Date Fee in Lieu Received',
 'Calculated Fee In Lieu',
 'Units <= 30% MFI',
 'Units <= 40% MFI',
 'Units <= 50% MFI',
 'Units <= 60% MFI',
 'Units <= 65% MFI',
 'Units <= 80% MFI',
 'Market Rate Units',
 'RHDA ',
 'A&D',
 'PA Bond ',
 'CLT',
 'DDB',
 'ERC',
 'MDA',
 'MUD',
 'MUDB',
 'NBG',
 'PUDDA',
 'PUDDB',
 'RNY',
 'SMART',
 'SMART-GF',
 'TOD',
 'UNO',
 'VMU',
 'VOL',
 '2006 GO Bond ',
 '2013 GO Bond',
 'CDBG',
 'HOME',
 'HTF',
 'CIP',
 'UNO TF ',
 'Total City Funding ',
 'LIHTC ',
 'Property Management Company',
 'Phone Number',
 'Website ',
 'Longitude',
 'Lat

In [49]:
# Extracting the columns we need for the analysis
AHI_new = AHI_to_load[['Zip Code ','Unit Type', 'Tenure','Total City Funding ','Longitude','Latitude',]].copy()
AHI_new = AHI_new.dropna()
AHI_new.head()

,Zip Code,Unit Type,Tenure,Total City Funding,Longitude,Latitude
0,78722,Multifamily,Rental,"$2,000,000.00",-97.708414,30.282007
1,78745,Multifamily,Rental,"$500,000.00",-97.793427,30.198641
2,78704,Multifamily,Rental,"$300,000.00",-97.771020,30.229837
3,78704,Multifamily,Rental,"$1,009,502.00",-97.764224,30.255306
4,78704,Multifamily,Rental,"$2,250,000.00",-97.765039,30.254456


In [50]:
# Flitering the data based on 'Ownership' and 'Single Family'
#(AHI_new['Unit Type'] == "Single Family") & 
AHI=AHI_new.loc[(AHI_new['Tenure'] == "Ownership")]
                
AHI = AHI.reset_index()

In [51]:
gkey=api_keys.gkey
gmaps.configure(api_key=gkey)

In [52]:
locations = AHI[["Latitude", "Longitude"]].astype(float)

In [53]:
ints=locations["Latitude"]
for idx, val in enumerate(ints):
    if(val>31):
        print(idx, val)
        locations=locations.drop(idx)

893 3029439926.0


In [54]:
# Create a AHI Heatmap layer
ahi_layer = gmaps.symbol_layer(
    locations, fill_color='red', stroke_color='red', scale=2
)
fig = gmaps.figure()
fig.add_layer(ahi_layer)
fig
embed_minimal_html('AHI_Map.html', views=[fig])

# Austin Housing Income and Price Data

In [55]:
# File to Load
AustinHousingIncomeAndPriceData = "Resources/Austin Housing Income and Price Data.csv"
# Read the Affordable Housing Inventory Data
AustinHousingIncomeAndPriceData = pd.read_csv("Resources/Austin Housing Income and Price Data.csv")
AustinHousingIncomeAndPriceData.head()

,Zip Code,Population below poverty level,Median household income,"Non-White, Non-Hispanic or Latino","Hispanic or Latino, of any race",Population with disability,Unemployment,Large households (5+ members),"Homes affordable to people earning less than $50,000","Rentals affordable to people earning less than $25,000",...,Rental units affordable to average tech worker,Owner units affordable to average tech worker,"Change in percentage of population below poverty, 2000-2012","Change in median rent, 2000-2012","Change in median home value, 2000-2012",Percentage of homes within 1/4-mi of transit stop,Average monthly transportation cost,Percentage of housing and transportation costs that is transportation-related,Longitude,Latitude
0,78726,9%,"66,096",16%,20%,3%,4%,2%,0%,1%,...,99%,17%,375%,20%,55%,2%,786,40%,-97.839500,30.419100
1,78724,38%,"35,711",29%,63%,8%,8%,28%,85%,16%,...,100%,96%,208%,37%,46%,21%,786,40%,-97.738824,30.256483
2,78617,18%,"43,957",12%,67%,10%,15%,23%,NaN,11%,...,99%,NaN,101%,74%,21%,16%,865,42%,-97.622300,30.294400
3,78701,20%,"68,152",16%,14%,10%,9%,0%,7%,7%,...,90%,30%,12%,115%,59%,97%,433,23%,-97.738824,30.256483
4,78702,33%,"34,734",18%,56%,14%,11%,10%,21%,41%,...,99%,67%,3%,73%,207%,96%,590,39%,-97.716349,30.267095


In [56]:
#list(AustinHousingIncomeAndPriceData.columns.values)

In [57]:
# Extracting the columns we need for the analysis
AustinHousingIncomeAndPriceData_new = AustinHousingIncomeAndPriceData[['Zip Code','Median home value','Longitude','Latitude',]].copy()
AustinHousingIncomeAndPriceData_new = AustinHousingIncomeAndPriceData_new.dropna()
AustinHousingIncomeAndPriceData_new

,Zip Code,Median home value,Longitude,Latitude
0,78726,"349,900",-97.839500,30.419100
1,78724,"105,700",-97.738824,30.256483
2,78617,"100,600",-97.622300,30.294400
3,78701,"338,300",-97.738824,30.256483
4,78702,"175,400",-97.716349,30.267095
5,78703,"621,900",-97.765120,30.281334
6,78704,"338,200",-97.763241,30.242616
7,78705,"292,500",-97.746225,30.289585
8,78717,"243,200",-97.780345,30.476253
9,78721,"121,000",-97.687691,30.274319


In [58]:
locations = AustinHousingIncomeAndPriceData_new[["Latitude", "Longitude"]].astype(float)
locations
median_hv = AustinHousingIncomeAndPriceData_new["Median home value"]

In [61]:
sum=0
arr=[]
for idx,val in enumerate(median_hv):
    val = int(val.replace(',',''))
    sum= sum+val
    
for idx,val in enumerate(median_hv):
    val = int(val.replace(',',''))
    avg= val/sum
    arr.append(avg*1000)


arr= pd.DataFrame(arr).astype(float)

In [60]:
# Create a AHIPD Heatmap layer

fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=arr,point_radius=20))
embed_minimal_html('AHIPD_Map.html', views=[fig])